In [ ]:
from lec_utils import *
def show_grouping_animation():
    src = "https://docs.google.com/presentation/d/1tBaFyHseIGsX5wmE3BdNLeVHnKksQtpzLhHge8Tzly0/embed?start=false&loop=false&delayms=60000&rm=minimal"
    width = 960
    height = 509
    display(IFrame(src, width, height))
def show_merging_animation():
    src = "https://docs.google.com/presentation/d/1HPJ7fiBLNEURsWYiY0qpqPR3qup68Mr0_B34GU99Y8Q/embed?start=false&loop=false&delayms=60000&rm=minimal"
    width = 865
    height = 509
    display(IFrame(src, width, height))

<div class="alert alert-info" markdown="1">

#### Lecture 6

# Grouping, Pivoting, and Merging

### EECS 398-003: Practical Data Science, Fall 2024

<small><a style="text-decoration: none" href="https://practicaldsc.org">practicaldsc.org</a> • <a style="text-decoration: none" href="https://github.com/practicaldsc/fa24">github.com/practicaldsc/fa24</a></small>
    
</div>

### Announcements 📣

- Homework 2 is due **tomorrow night** – we've provided a 24-hour extension to everyone since we released it a bit late. Don't be discouraged if you feel like there's a lot of reading or you have to do a lot of Googling – this is intentional, because this is how data scientists actually have to solve problems!
<br><small>Post on [Ed](https://edstem.org/us/courses/61012/discussion/) or
come to [Office Hours](https://practicaldsc.org/calendar) for help! We're using a queue for office hours now – access it from [practicaldsc.org/calendar](https://practicaldsc.org/calendar).</small>

- [**study.practicaldsc.org**](https://study.practicaldsc.org) contains our discussion worksheets (and solutions), which are made up of old exam problems. Use these problems to build your theoretical understanding of the material!

- Homework 1 scores are available on Gradescope.<br><small>Now, you can even see which hidden tests you failed.</small>

### Agenda

- Recap: `groupby`.
- Advanced `groupby` usage.
- Pivot tables using the `pivot_table` method.
- Merging.

Remember to follow along in lecture by accessing the "blank" lecture notebook in our [public GitHub repository](https://github.com/practicaldsc/fa24).

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>
    
We have 2 office hours on Central Campus each week, but not many students have attended. What should we do with them?
    
- A. Keep them – I'll come at some point.
- B. Keep them on Central Campus, but change them to a different time (let us know when in the free response box).
- C. Replace them with more North Campus Office hours (let us know when in the free response box).

## Recap: `groupby`

Recall, we use `groupby` when we need to calculate something **for each** group.

---

In [ ]:
show_grouping_animation()

Run the cell below to load in our dataset.

In [ ]:
penguins = sns.load_dataset('penguins').dropna().reset_index(drop=True)
penguins

### Example: Finding the mean `'bill_length_mm'` of each species

- If you were asked to find the mean `'bill_length_mm'` of `'Adelie'` penguins, you could do so using a query.

In [ ]:
penguins.loc[penguins['species'] == 'Adelie', 'bill_length_mm'].mean() 

- But, you can find the mean `'bill_length_mm'` of all three species using `groupby`:

In [ ]:
# Read this as:
#        for each 'species',  calculate the mean 'bill_length_mm'.
penguins.groupby('species')['bill_length_mm'].mean() 

### Understanding the syntax of `groupby`

- <code style="background: none"><strong><span style="color: #0066cc">penguins.groupby('species')</span></strong><span style="color: #999">['bill_length_mm'].mean()</span></code><br>First, tell `pandas` which column you want to group by. Since we're grouping by `'species'`, the remainder of the calculations will be done separately for each `'species'`.

- <code style="background: none"><span style="color: #999">penguins.groupby('species')</span><strong><span style="color: #0066cc">['bill_length_mm']</span></strong><span style="color: #999">.mean()</span></code><br>Then, select the other column(s) that you want to aggregate. Here, we want to calculate mean `'bill_length_m'`s, so that's what we select.</span>

- <code style="background: none"><span style="color: #999">penguins.groupby('species')['bill_length_mm']</span><strong><span style="color: #0066cc">.mean()</span></strong></code><br>Finally, we use an aggregation method. This is saying, for each `'species'`, compute the mean `'bill_length_mm'`.

## Advanced `groupby` usage

---

### Beyond default aggregation methods

- There are many built-in aggregation methods.

- What if you want to apply different aggregation methods to different columns?

- What if the aggregation method you want to use doesn't already exist in `pandas`?

### The `aggregate` method

- `DataFrameGroupBy` and `SeriesGroupBy` objects have a general `aggregate` method, which aggregates using one or more operations.<br><small>Remember, aggregation is the act of combining many values into a single value.</small>

- There are many ways of using `aggregate`; refer to [the documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.DataFrameGroupBy.aggregate.html) for a comprehensive list.<br><small>Per [the documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.agg.html), `agg` is an alias for `aggregate`.</small>

- Example arguments:
    - A single function.
    - A list of functions.
    - A dictionary mapping column names to functions.

- We've attached a Reference Slide with examples.

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Examples

- How many penguins are there of each `'species'`, and what is the mean `'body_mass_g'` of each `'species'`?

In [ ]:
(
    penguins
    .groupby('species')
    ['body_mass_g']
    .aggregate(['count', 'mean'])
)

- What is the maximum `'bill_length_mm'` of each `'species'`, and which `'island'`s is each `'species'` found on?

In [ ]:
(
    penguins
    .groupby('species')
    .agg({'bill_length_mm': 'max', 'island': 'unique'})
)

<div class="alert alert-success">
<h3>Activity</h3>

What is the **interquartile range** of the `'body_mass_g'` of each `'species'`?
    
The interquartile range of a distribution is defined as:
    
$$\text{75th percentile} - \text{25th percentile}$$
    
***Hint***: Use `np.percentile`, and pass `agg`/`aggregate` a custom function.

In [ ]:
# Here, the argument to agg is a function,
# which takes in a Series and returns a scalar.
def iqr(s):
    return np.percentile(s, 75) - np.percentile(s, 25)
(
    penguins
    .groupby('species')
    ['body_mass_g']
    .agg(iqr)
)

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>
    
What questions do you have?

### Split-apply-combine, revisited

- When we introduced the split-apply-combine pattern, the "apply" step involved **aggregation** – our final DataFrame had one row for each group.

<center><img src="imgs/image_0.png" width=40%></center>

- Instead of aggregating during the apply step, we could instead perform a **filtration**, in which we keep only the groups that satisfy some condition.

- Or a **transformation**, in which we perform operations to every value within each group.

### Grouping, then filtering

- To keep only the groups that satisfy a particular condition, use the `filter` method on a `DataFrameGroupBy`/`SeriesGroupBy` object.<br><small>The `filter` method takes in a function, which itself takes in a DataFrame/Series and return a single Boolean. The result is a new DataFrame/Series with only the groups for which the filter function returned `True`.</small>

- For example, suppose we want only the `'species'` whose average `'bill_length_mm'` is above 39.

In [ ]:
(
    penguins
    .groupby('species')
    .filter(lambda df: df['bill_length_mm'].mean() > 39)
)

- No more `'Adelie'`s!

<div class="alert alert-success">
<h3>Activity</h3>

Create a new DataFrame with only the rows in `penguins` for popular `'species'` – that is, `'species'` with at least 100 penguins.

In [ ]:
(
    penguins
    .groupby('species')
    .filter(lambda df: df.shape[0] >= 100)
)

In [ ]:
# Note that to just find the 'species' with at least 100 penguins,
# we didn't need to group:
penguins['species'].value_counts()

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Example: Z-Scoring

- Suppose we want to convert the `'body_mass_g'` column to to z-scores (i.e. standard units):

$$z(x_i) = \frac{x_i - \text{mean of } x}{\text{SD of } x}$$

In [ ]:
def z_score(x):
    return (x - x.mean()) / x.std(ddof=0)

In [ ]:
z_score(penguins['body_mass_g'])

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Grouping, then transforming

- Now, what if we wanted the z-score within each group?

- To do so, we can use the `transform` method on a `DataFrameGroupBy` object. The `transform` method takes in a function, which itself takes in a Series and returns a new Series.

- A transformation produces a DataFrame or Series of the same size – it is **not** an aggregation!

In [ ]:
z_mass = (penguins
          .groupby('species')
          ['body_mass_g']
          .transform(z_score))
z_mass

In [ ]:
penguins.assign(z_mass=z_mass)

In [ ]:
display_df(penguins.assign(z_mass=z_mass), rows=8)

- Note that above, penguin 340 has a larger `'body_mass_g'` than penguin 0, but a lower `'z_mass'`.
    - Penguin 0 has an above average `'body_mass_g'` among `'Adelie'` penguins.
    - Penguin 340 has a below average `'body_mass_g'` among `'Gentoo'` penguins. Remember from earlier that the average `'body_mass_g'` of `'Gentoo'` penguins is much higher than for other species.

### Grouping with multiple columns

- When we group with multiple columns, one group is created for **every unique combination** of elements in the specified columns.<br><small>In the output below, why are there only 5 rows, rather than $3 \times 3 = 9$ rows, when there are 3 unique `'species'` and 3 unique `'island'`s?</small>

In [ ]:
# Read this as:
species_and_island = (
    penguins.groupby(['species', 'island'])         # for every combination of 'species' and 'island' in the DataFrame,
    [['bill_length_mm', 'bill_depth_mm']].mean()    # calculate the mean 'bill_length_mm' and the mean 'bill_depth_mm'.
)
species_and_island

- When grouping by multiple columns, the resulting DataFrame has a `MultiIndex`.

In [ ]:
species_and_island['bill_length_mm'] 

In [ ]:
species_and_island.loc['Adelie'] 

In [ ]:
species_and_island.loc[('Adelie', 'Torgersen')] 

- **Advice**: When working with a `MultiIndex`, use `reset_index` or set `as_index=False` in `groupby`.

In [ ]:
# Now, this looks like a regular DataFrame!
species_and_island.reset_index() 

<div class="alert alert-success">
<h3>Activity</h3>
        
Find the most popular <code>'Male'</code> and <code>'Female'</code> baby <code>'Name'</code> for each <code>'Year'</code> in <code>baby</code>. <b>Exclude</b> <code>'Year'</code>s where there were fewer than 1 million births recorded.
</div>

In [ ]:
baby = pd.read_csv('data/baby.csv')
baby

In [ ]:
(
    baby
    .groupby('Year')
    .filter(lambda df: df['Count'].sum() >= 1_000_000) # Keeps only the 'Year's with at least 1,000,000 births.
    .sort_values('Count', ascending=False)             # Sorts by 'Count' in descending order, so the most popular 'Name's are always at the top.
    .groupby(['Year', 'Sex'])                          # Finds the first row for every combination of ('Year', 'Sex').
    .first()
)

## Pivot tables using the `pivot_table` method

---

### Pivot tables: An extension of grouping

- Pivot tables are a compact way to display tables for humans to read:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Sex</th>
      <th>F</th>
      <th>M</th>
    </tr>
    <tr>
      <th>Year</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>2018</th>
      <td>1698373</td>
      <td>1813377</td>
    </tr>
    <tr>
      <th>2019</th>
      <td>1675139</td>
      <td>1790682</td>
    </tr>
    <tr>
      <th>2020</th>
      <td>1612393</td>
      <td>1721588</td>
    </tr>
    <tr>
      <th>2021</th>
      <td>1635800</td>
      <td>1743913</td>
    </tr>
    <tr>
      <th>2022</th>
      <td>1628730</td>
      <td>1733166</td>
    </tr>
  </tbody>
</table>

- Notice that each value in the table is a sum of the `'Count'`s, for different combinations of `'Year'` and `'Sex'`.

- **You can think of pivot tables as grouping using two columns, then "pivoting" one of the group labels into columns.**

### `pivot_table`

- The `pivot_table` DataFrame method aggregates a DataFrame using two columns. To use it:
<br><br>
```python
        df.pivot_table(index=index_col,
                       columns=columns_col,
                       values=values_col,
                       aggfunc=func)
```

- The resulting DataFrame will have:
    - One row for every unique value in `index_col`.
    - One column for every unique value in `columns_col`.
    - Values determined by applying `func` on values in `values_col`.

In [ ]:
last_5_years = baby[baby['Year'] >= 2018] 
last_5_years

In [ ]:
last_5_years.pivot_table(
    index='Year',
    columns='Sex',
    values='Count',
    aggfunc='sum',
)

In [ ]:
# Same information as above, but harder to read!
(
    last_5_years
    .groupby(['Year', 'Sex'])
    [['Count']]
    .sum()
)

### Example: Finding the number of penguins per `'island'` and `'species'`

- As a refresher, the `penguins` DataFrame looks like this:

In [ ]:
penguins

- Suppose we want to find the number of penguins in `penguins` per `'island'` and `'species'`. We can do so _without_ `pivot_table`:

In [ ]:
penguins.value_counts(['island', 'species']) 

In [ ]:
penguins.groupby(['island', 'species']).size() 

- But the data is arguably easier to interpret when we do use `pivot_table`:

In [ ]:
penguins.pivot_table(
    index='species', 
    columns='island', 
    values='bill_length_mm', # Choice of column here doesn't actually matter! Why?
    aggfunc='count',
)

- Note that there is a `NaN` at the intersection of `'Biscoe'` and `'Chinstrap'`, because there were no Chinstrap penguins on Biscoe Island.<br><small>`NaN` stands for "not a number." It is `numpy` and `pandas`' version of a null value (the regular Python null value is `None`). We'll learn more about how to deal with these soon.</small>

- We can either use the `fillna` method afterwards or the `fill_value` argument to fill in `NaN`s.

In [ ]:
penguins.pivot_table(
    index='species', 
    columns='island', 
    values='bill_length_mm', 
    aggfunc='count',
    fill_value=0,
)

### Granularity, revisited

- Each row of the original `penguins` DataFrame represented a single penguin, and each column represented features of the penguins.

In [ ]:
penguins

- What is the granularity of the DataFrame below?<br><small>That is, what does each row represent?</small>

In [ ]:
penguins.pivot_table(
    index='species', 
    columns='island', 
    values='bill_length_mm', 
    aggfunc='count',
    fill_value=0,
)

### Reshaping

- `pivot_table` reshapes DataFrames from "long" to "wide".

- Other DataFrame reshaping methods:
    - `melt`: Un-pivots a DataFrame. Very useful in data cleaning.
    - `pivot`: Like `pivot_table`, but doesn't do aggregation.
    - `stack`: Pivots multi-level columns to multi-indices.
    - `unstack`: Pivots multi-indices to columns.

- Google, the documentation, and ChatGPT are your friends!

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>
    
What questions do you have?

## Merging

---

In [ ]:
phones = pd.DataFrame().assign(
    Model=['iPhone 16', 'iPhone 16 Pro Max', 'Samsung Galaxy S24 Ultra', 'Pixel 9 Pro'],
    Price=[799, 1199, 1299, 999],
    Screen=[6.1, 6.9, 6.8, 6.3]
)
inventory = pd.DataFrame().assign(
    Handset=['iPhone 16 Pro Max', 'iPhone 16', 'Pixel 9 Pro', 'Pixel 9 Pro', 'iPhone 16', 'iPhone 15'],
    Units=[50, 40, 10, 15, 100, 5],
    Store=['Briarwood', 'Somerset', 'Arbor Hills', '12 Oaks', 'Briarwood', 'Oakland Mall']
)

### Example: Phone sales 📱

In [ ]:
# The DataFrame on the left contains information about phones on the market.
# The DataFrame on the right contains information about the stock I have in my stores.
dfs_side_by_side(phones, inventory)

- **Question**: If I sell all of the phones in my inventory, how much will I make in revenue?

- The information I need to answer the question is spread across multiple DataFrames.

- The solution is to **merge** the two DataFrames together.<br><small>The SQL term for merge is **join**.</small>

- A merge is appropriate when we have two sources of information **about the same individuals** that is **linked by a common column(s)**.The common column(s) are called the **join key**.<br>

### If I sell all of the phones in my inventory, how much will I make in revenue?

In [ ]:
combined = phones.merge(inventory, left_on='Model', right_on='Handset') 
combined

In [ ]:
np.sum(combined['Price'] * combined['Units']) 

### What just happened!? 🤯

In [ ]:
# Click through the presentation that appears.
show_merging_animation()

### The `merge` method

- The `merge` DataFrame method joins two DataFrames by columns or indexes.<br><small>As mentioned before, "merge" is just the `pandas` word for "join."</small>

- When using the `merge` method, the DataFrame before `merge` is the "left" DataFrame, and the DataFrame passed into `merge` is the "right" DataFrame.<br><small>In `phones.merge(inventory)`, `phones` is considered the "left" DataFrame and `inventory` is the "right" DataFrame.<br>The columns from the left DataFrame appear to the left of the columns from right DataFrame.</small>

- By default:
    - If join keys are not specified, all shared columns between the two DataFrames are used.
    - The "type" of join performed is an inner join, which is just one of many types of joins.

### Inner joins

- The default type of join that `merge` performs is an **inner join**, which keeps the **intersection** of the join keys.

<center><img src='imgs/inner-joins.png' width=300></center>

In [ ]:
# The DataFrame on the far right is the merged DataFrame.
dfs_side_by_side(phones, inventory, phones.merge(inventory, left_on='Model', right_on='Handset'))

- Note that `'Samsung Galaxy S24 Ultra'` and `'iPhone 15'` do not appear in the merged DataFrame.

- That's because there is no `'Samsung Galaxy S24 Ultra'` in the right DataFrame (`inventory`), and no `'iPhone 15'` in the left DataFrame (`phones`).

### Other join types

- We can change the type of join performed by changing the `how` argument in `merge`.

In [ ]:
phones.merge(inventory, left_on='Model', right_on='Handset', how='left')

In [ ]:
phones.merge(inventory, left_on='Model', right_on='Handset', how='right')

In [ ]:
phones.merge(inventory, left_on='Model', right_on='Handset', how='outer')

- The website [pandastutor.com](https://pandastutor.com) can help visualize DataFrame operations like these. [**Here's a direct link**](https://pandastutor.com/vis.html#code=%23%20The%20code%20below%20just%20initializes%20the%0A%23%20phones%20and%20inventory%20DataFrames%20as%20we've%20seen%20them%0A%23%20in%20lecture.%0A%23%20---%0Aimport%20pandas%20as%20pd%0Aimport%20io%0A%0Aphones%20%3D%20pd.read_csv%28io.StringIO%28'''%0AModel,Price,Screen%0AiPhone%2016,799,6.1%0AiPhone%2016%20Pro%20Max,1199,6.9%0ASamsung%20Galaxy%20S24%20Ultra,1299,6.8%0APixel%209%20Pro,999,6.3%0A'''%29%29%0A%0Ainventory%20%3D%20pd.read_csv%28io.StringIO%28'''%0AHandset,Units,Store%0AiPhone%2016%20Pro%20Max,50,Briarwood%0AiPhone%2016,40,Somerset%0APixel%209%20Pro,10,Arbor%20Hills%0APixel%209%20Pro,15,12%20Oaks%0AiPhone%2016,100,Briarwood%0AiPhone%2015,5,Oakland%20Mall%0A'''%29%29%0A%0A%23%20---%0A%0A%23%20Below,%20uncomment%20the%20line%20that%20you'd%20like%20to%20visualize!%0A%23%20phones.merge%28inventory,%20left_on%3D'Model',%20right_on%3D'Handset',%20how%3D'inner'%29%0A%0Aphones.merge%28inventory,%20left_on%3D'Model',%20right_on%3D'Handset',%20how%3D'left'%29%0A%0A%23%20phones.merge%28inventory,%20left_on%3D'Model',%20right_on%3D'Handset',%20how%3D'right'%29%0A%0A%23%20phones.merge%28inventory,%20left_on%3D'Model',%20right_on%3D'Handset',%20how%3D'outer'%29&d=2024-09-11&lang=py&v=v1) to this specific example.

<center><img src="imgs/pt-screenshot.png" width=800></center>

### Different join types handle mismatches differently

<center><img src='imgs/all-joins.png' width=400></center>

- **Inner join**: Keep **only** the matching keys (intersection).

- **Outer join**: Keep **all** keys in both DataFrames (union).

- **Left join**: Keep all keys in the left DataFrame, whether or not they are in the right DataFrame.

- **Right join**: Keep all keys in the right DataFrame, whether or not they are in the left DataFrame.<br><small> Note that `a.merge(b, how='left')` contains the same information as `b.merge(a, how='right')`, just in a different order.</small>

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Notes on the `merge` method

- `merge` is flexible – you can merge using a combination of columns, or the index of the DataFrame.

-  If the two DataFrames have the same column names, `pandas` will add `_x` and `_y` to the duplicated column names to avoid having columns with the same name (change these the `suffixes` argument).

- There is, in fact, a `join` method, but it's actually a wrapper around `merge` with fewer options.

- **As always, the [documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) is your friend!**

<div class="alert alert-danger" markdown="1">

#### Reference Slide

### Lots of `pandas` operations do an implicit outer join!

- `pandas` will almost always try to match up index values using an outer join.

- It won't tell you that it's doing an outer join, it'll just throw `NaN`s in your result!

In [ ]:
df1 = pd.DataFrame({'a': [1, 2, 3]}, index=['hello', 'eecs398', 'students'])
df2 = pd.DataFrame({'b': [10, 20, 30]}, index=['eecs398', 'is', 'awesome'])
dfs_side_by_side(df1, df2)

In [ ]:
df1['a'] + df2['b']

### Activity setup

In [ ]:
midwest_cities = pd.DataFrame().assign(
    city=['Ann Arbor', 'Detroit', 'Chicago', 'East Lansing'],
    state=['Michigan', 'Michigan', 'Illinois', 'Michigan'],
    today_high_temp=['79', '83', '87', '87']
)
schools = pd.DataFrame().assign(
    name=['University of Michigan', 'University of Chicago', 'Wayne State University', 'Johns Hopkins University', 'UC San Diego', 'Concordia U-Ann Arbor', 'Michigan State University'], 
    city=['Ann Arbor', 'Chicago', 'Detroit', 'Baltimore', 'La Jolla', 'Ann Arbor', 'East Lansing'],
    state=['Michigan', 'Illinois', 'Michigan', 'Maryland', 'California', 'Michigan', 'Michigan'],
    graduation_rate=[0.87, 0.94, 0.78, 0.92, 0.81, 0.83, 0.91]
)

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a style="text-decoration: none; color: #0066cc" href="https://docs.google.com/forms/d/e/1FAIpQLSd4oliiZYeNh76jWy-arfEtoAkCrVSsobZxPwxifWggo3EO0Q/viewform">practicaldsc.org/q</a>)</h3>
    
<small>Remember that you can always ask questions anonymously at the link above!</small>

**Without writing code**, how many rows are in `midwest_cities.merge(schools, on='city')`?

<br>
    
<center>
A. <b>4</b> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    B. <b>5</b> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    C. <b>6</b> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    D. <b>7</b> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;    E. <b>8</b></center>

In [ ]:
dfs_side_by_side(midwest_cities, schools)

In [ ]:
# Answer: 5.
midwest_cities.merge(schools, on='city')

### Followup activity

**Without writing code**, how many rows are in `midwest_cities.merge(schools, on='state')`?

In [ ]:
dfs_side_by_side(midwest_cities, schools)

In [ ]:
# Answer: 13.
midwest_cities.merge(schools, on='state')

<div class="alert alert-success">
<h3>Activity</h3>
        
Fill in the blank so that the last statement evaluates to `True`.

```python
df = midwest_cities.merge(schools, on='state')
df.shape[0] == (____).sum()
```

**Don't** use `merge` (or `join`) in your solution!


In [ ]:
midwest_cities['state'].value_counts()

In [ ]:
schools['state'].value_counts()

In [ ]:
# When we multiply the above two Series,
# the product is done by matching up the index.
midwest_cities['state'].value_counts() * schools['state'].value_counts()

In [ ]:
# When we sum the resulting Series, the missing values are ignored.
# The expression below evaluates to 13, which is the answer to the previous slide's question.
(midwest_cities['state'].value_counts() * schools['state'].value_counts()).sum()

### What's next?

- How do we decide which type of visualization to create?

- How do we deal with missing values?

<div class="alert alert-danger" markdown="1">

#### Reference Section

## Another example

The rest of the notebook contains an additional example of how `merge` can be used to solve larger problems. It's a good idea to walk through it as if it's an exercise. Try to fill in the missing code here, and look at the posted HTML on the course website (or `lec06-filled.ipynb`) for solutions.

---

### Name categories

- This [New York Times article](https://archive.is/NpORG) claims that certain categories of names are becoming more popular. For example:

    - Forbidden names like Lucifer, Lilith, Kali, and Danger.

    - Evangelical names like Amen, Savior, Canaan, and Creed.

    - Mythological names.

    - It also claims that baby boomer names are becoming less popular.

- Let's see if we can verify these claims using data!

### Loading in the data

- Our first DataFrame, `baby`, is the same as we saw earlier in the lecture. It has one row for every combination of `'Name'`, `'Sex'`, and `'Year'`.

In [ ]:
baby

- Our second DataFrame, `nyt`, contains the New York Times' categorization of each of several names, based on the aforementioned article.

In [ ]:
nyt = pd.read_csv('data/nyt_names.csv')
nyt

- **Issue**: To find the number of babies born with (for example) forbidden names each year, we need to combine information from both `baby` and `nyt`.

- **Solution**: `merge` the two DataFrames!

### Returning back to our original question

**Your Job**: Assign `category_counts` to a DataFrame that contains one row for every combination of `'category'` in `nyt` and `'Year'` in `baby`. It should have three columns: `'category'`, `'Year'`, and `'Count'`, where `'Count'` contains the total number of babies born with that name `'category'` in that `'Year'`. The first few rows of the DataFrame you're meant to create are given below.

In [ ]:
category_counts = ...
category_counts = (
    baby
    .merge(nyt, left_on='Name', right_on='nyt_name')
    .groupby(['category', 'Year'])
    ['Count']
    .sum()
    .reset_index()
)
category_counts

Once you've done that, run the cell below!

In [ ]:
# We'll talk about plotting code soon!
import plotly.express as px
fig = px.line(category_counts, x='Year', y='Count',
              facet_col='category', facet_col_wrap=3,
              facet_row_spacing=0.15,
              width=600, height=400)
fig.update_yaxes(matches=None, showticklabels=False)